*This notebook is adapted from the [Unsloth Gemma 2 2B Training Notebook](https://colab.research.google.com/drive/1weTpKOjBZxZJ5PQ-Ql8i6ptAY2x-FWVA?usp=sharing) tutorial. Go support them!*

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a><br>Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth#installation-instructions---conda).

In [ ]:
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!apt-get install build-essential -y
!pip install huggingface_hub wandb
!pip uninstall torch -y
!pip install torch==2.3.0
!pip install -U optimum
!pip uninstall xformers -y
!pip install "xformers<0.0.27" --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!wandb login 1bedcc01276765f024f9379a6a681a16c5b4c7cd

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1248 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b-it-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: If you want to finetune Gemma 2, install flash-attn to make it faster!
To install flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
==((====))==  Unsloth 2024.8: Fast Gemma2 patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.691 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = False, # True or "unsloth" for very long context
    random_state = 4249,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


In [3]:
from datasets import load_dataset

template = """<bos><start_of_turn>user
{human}<end_of_turn>
<start_of_turn>model
{gpt}<|eot_id|>"""

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [template.format(human=convo[0]["value"], gpt=convo[1]["value"]) for convo in convos]
    return { "text" : texts, }
pass

dataset = load_dataset('gsarti/eureka-rebus', 'llm_sft', data_files=["train.jsonl"], split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True)

In [4]:
print(dataset[5]["text"])

<bos><start_of_turn>user
Risolvi gli indizi tra parentesi per ottenere una prima lettura, e usa la chiave di lettura per ottenere la soluzione del rebus.

Rebus: U [Lo è il passacavallo] LO [È fatta di vimini] F F [Sono decimi di chilo] S [Disusato soprabito] A [Un rampicante dei Tropici]
Chiave di lettura: 3 6 12 8<end_of_turn>
<start_of_turn>model
Procediamo alla risoluzione del rebus passo per passo:
- U = U
- [Lo è il passacavallo] = nave
- L O = L O
- [È fatta di vimini] = cesta
- F F = F F
- [Sono decimi di chilo] = etti
- S = S
- [Disusato soprabito] = tait
- A = A
- [Un rampicante dei Tropici] = liana

Prima lettura: U nave LO cesta F F etti S tait A liana

Ora componiamo la soluzione seguendo la chiave risolutiva:
3 = Una
6 = veloce
12 = staffettista
8 = italiana

Soluzione: Una veloce staffettista italiana
<|eot_id|>


In [5]:
eval_dataset = load_dataset('gsarti/eureka-rebus', 'llm_sft', data_files=["id_test.jsonl", "ood_test.jsonl"], split = "train")
eval_dataset = eval_dataset.map(formatting_prompts_func, batched = True)

In [6]:
print(eval_dataset[5]["text"])

<bos><start_of_turn>user
Risolvi gli indizi tra parentesi per ottenere una prima lettura, e usa la chiave di lettura per ottenere la soluzione del rebus.

Rebus: [Può avere il campanile] VIO [Possedevano la bacchetta magica] SO R [Componimento lirico] LL [Si innamorò di Narciso] [Il valore del suffisso"esa"] M [Gomma per suole] TE
Chiave di lettura: 3 1 5 2 6 5 4 8<end_of_turn>
<start_of_turn>model
Procediamo alla risoluzione del rebus passo per passo:
- [Può avere il campanile] = chiesa
- V I O = V I O
- [Possedevano la bacchetta magica] = fate
- S O R = S O R
- [Componimento lirico] = ode
- L L = L L
- [Si innamorò di Narciso] = eco
- [Il valore del suffisso"esa"] = sei
- M = M
- [Gomma per suole] = para
- T E = T E

Prima lettura: chiesa VIO fate SO R ode LL eco sei M para TE

Ora componiamo la soluzione seguendo la chiave risolutiva:
3 = Chi
1 = è
5 = savio
2 = fa
6 = tesoro
5 = delle
4 = cose
8 = imparate

Soluzione: Chi è savio fa tesoro delle cose imparate
<|eot_id|>


In [7]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

response_template = "<start_of_turn>model"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    data_collator=collator,
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 4,
        warmup_steps = 100,
        num_train_epochs = 2,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 100,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        output_dir = "outputs",
        lr_scheduler_type = "linear",
        seed = 42,
        save_steps = 500,
        save_total_limit = 10,
        eval_strategy = "steps",
        eval_steps = 500,
        push_to_hub = True,
        hub_model_id = "gemma-2-2b-rebus-solver-adapters",
        report_to="wandb"
    ),
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [8]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3090. Max memory = 23.691 GB.
2.57 GB of memory reserved.


In [9]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 81,136 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 4
\        /    Total batch size = 32 | Total steps = 5,070
 "-____-"     Number of trainable parameters = 20,766,720
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gsarti. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
500,0.066800,0.091066
1000,0.041500,0.071861
1500,0.031400,0.062541
2000,0.025000,0.057362
2500,0.020100,0.054294
3000,0.010200,0.056236
3500,0.009800,0.054726
4000,0.008400,0.053961
4500,0.008100,0.052070
5000,0.007300,0.050959


In [10]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

10943.3072 seconds used for training.
182.39 minutes used for training.
Peak reserved memory = 22.186 GB.
Peak reserved memory for training = 19.616 GB.
Peak reserved memory % of max memory = 93.647 %.
Peak reserved memory for training % of max memory = 82.799 %.


In [ ]:
# Save merged model in FP16
model.push_to_hub_merged("gsarti/gemma-2-2b-rebus-solver-fp16", tokenizer, save_method = "merged_16bit")

# Save model to GGUF for local inference
model.push_to_hub_gguf("gsarti/gemma-2-2b-rebus-solver-Q8_0-GGUF", tokenizer)